# Estimasi Tropical Cyclone Intensity menggunakan Deep  Convolutional Neural Network - bagian 2 

**isi pada notebook:**

- [Memahami Persyaratan Model](#Memahami-Persyaratan-Model)
    - [Menjelajahi Opsi Pengubahan Ukuran](#Menjelajahi-berbagai-jenis-opsi-pengubahan-ukuran)
    - [Memilih Patch Acak](#Langkah-2-:-Memilih-Patch-Acak-dari-Gambar)
- [Menganotasi Kumpulan Data ](#Menganotasi-Kumpulan-Data) 
- [Penggabungan](#Penggabungan-:)
    - [Mempersiapkan Dataset](#Mempersiapkan-Dataset)
    - [Mendefinisikan Model](#Mendefinisikan-Model)
    - [Menyusun dan Melatih Model](#Menyusun-dan-Melatih-Model)
    - [Visualisasi](#Visualisasi)

**Pada akhir Notebook ini Anda akan:**

- Memahami Persyaratan Model.
- Anotasi Dataset.
- Latih Model Anda.

## Memahami Persyaratan Model

### Kita telah melihat model yang akan diproses pada gambar kita

- Model yang dijelaskan di Paper 
![alt text](images/model.png)

Kita dapat melihat bahwa gambar harus dalam bentuk ( 232, 232, 3) untuk dimasukkan ke dalam model kita.

Jadi, kita akan melakukan langkah-langkah berikut sebelum memasukkan gambar ke dalam model kita.

- Langkah 1 : Ubah Ukuran Gambar dari ( 1024, 1024 ,3) menjadi ( 256 , 256 ,3 ) 
- Langkah 2 : Pilih acak ( 232 , 232 , 3 ) dari patch ( 256 , 256 , 3 ) dan masukkan ke dalam model kita.

**Pendekatan Alternatif** : Kita dapat memodifikasi bentuk input model menjadi ( 256 x 256 x 3 ) dan melatihnya pada gambar yang diskalakan, tetapi kita mengambil tambalan acak ( 232 x 232 x 3 ) sehingga model kita tidak berharap siklon berada di tengah dan belajar memahami pemetaan bahkan dengan siklon di sisi gambar.

### Langkah 1 :
Sekarang mari mulai dengan Langkah 1 dan pahami semua metode pengubahan ukuran yang tersedia untuk melakukannya.

In [ ]:
import cv2
#Membaca Gambar dengan menggunakan cv2.imread()
img = cv2.imread('images/image_shape.jpg',1)
#Mengubah Ruang Warna
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# Cetak Bentuk Gambar
img.shape

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
#Plot Gambar
plt.imshow(img)

## Menjelajahi berbagai jenis opsi pengubahan ukuran 

Gambar dapat diubah ukurannya dengan berbagai cara. Beberapa metode adalah sebagai berikut (seperti yang dinyatakan dalam dokumentasi OpenCV) : 

<h3>Penskalaan</h3>
<p>Penskalaan hanyalah mengubah ukuran gambar. OpenCV dilengkapi dengan fungsi <b>cv2.resize()</a></b> untuk tujuan ini. Ukuran gambar dapat ditentukan secara manual, atau Anda dapat menentukan faktor penskalaan. Metode interpolasi yang berbeda digunakan. Metode interpolasi yang lebih disukai adalah <b>cv2.INTER_AREA</b> untuk penyusutan dan <b>cv2.INTER_CUBIC</b> (lambat) &amp; <b>cv2.INTER_LINEAR</b> untuk memperbesar. Secara default, metode interpolasi yang digunakan adalah <b>cv2.INTER_LINEAR</b> untuk semua tujuan pengubahan ukuran.

* cv2.INTER_AREA    ( Lebih digunakan untuk Menyusut ) 
* cv2.INTER_CUBIC   ( Lebih digunakan untuk Zooming tetapi lambat )
* cv2.INTER_LINEAR  ( Lebih digunakan untuk Zooming dan opsi default )


### Langkah 2 : Memilih Patch Acak dari Gambar

Kita akan menggunakan fungsi `np.random.randint()` dari toolbox Numpy untuk menghasilkan angka acak. Parameter fungsi ini adalah batas atas dan ukuran larik Keluaran seperti yang disebutkan dalam [Numpy Documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html)

## Wrapping things up (Penggabungan)

In [ ]:
#Import numpy untuk Menghasilkan Angka Acak
import numpy as np
#Membuat nomor acak(random) mulai dari [0,0] sampai [23,23] dan mendefinisikan titik awal sampai titik akhir 
start_pt= np.random.randint(24,size=2)
end_pt = start_pt + [232,232]
# Menskalakan gambar dan mengambil patch acak dari gambar tersebut
img  = cv2.resize(img,(256,256))
rand = img[start_pt[0]:end_pt[0],start_pt[1]:end_pt[1]]
plt.imshow(rand)

In [ ]:
rand.shape

Output Dari gambar akhir diperoleh sebagai (232, 232, 3)

# Menganotasi Kumpulan Data 

Mari kita mulai dengan mengambil contoh Badai Katrina dari tahun 2005 dan menskalakannya untuk semua Topan

In [ ]:
import pandas as pd
# Baca CSV yang kita simpan sebelumnya
df = pd.read_csv('atlantic_storms.csv')
# Buat Mask untuk Menyaring Topan Katrina kami (2005)
mask = (df['date'] > '2005-01-01') & (df['date'] <= '2006-01-01') & ( df['name'] == 'KATRINA')
# Terapkan Mask ke DataFrame Asli dan Ekstrak Dataframe Baru
new_df = df.loc[mask]

In [ ]:
new_df

In [ ]:
#Mendapatkan daftar Gambar dari Kumpulan Data Kita untuk Katrina
import os
e = os.listdir('Dataset/tcdat/tc05/ATL/12L.KATRINA/ir/geo/1km')
e.sort()
#Tampilkan lima gambar pertama
e[:5]

#### Dapat kita amati, gambar diambil setiap 30 menit sekali, tetapi data teks tersedia setiap 6 jam sekali. Jadi kita akan menginterpolasi data teks agar sesuai dengan kurva

In [ ]:
#Dapatkan daftar Tanggal dan Kecepatan dari Dataframe Baru
date_list = new_df['date'].tolist()
velocity_list = new_df['maximum_sustained_wind_knots'].tolist()

In [ ]:
print(date_list[:5])
type(date_list[0])

Tanggal dalam Format STR yang akan kita konversi sekarang ke format datetime untuk digunakan.

In [ ]:
from datetime import datetime
#Dapatkan dan Konversikan ke format Datetime untuk Waktu Rekaman Data Teks Pertama Terakhir.
first = (datetime.strptime(date_list[0], "%Y-%m-%d %H:%M:%S"))
last = (datetime.strptime(date_list[-1], "%Y-%m-%d %H:%M:%S"))
print(first)
type(first)

In [ ]:
#Mengubah daftar dari Konversi semuanya menjadi detik dari gambar pertama untuk menginterpolasi data
for i in range(len(date_list)):
    date_list[i]=( (datetime.strptime(date_list[i], "%Y-%m-%d %H:%M:%S")) - first ).total_seconds()
    print(date_list[i])

In [ ]:
# Interpolasi menggunakan Fungsi Perpustakaan Scipy
from scipy import interpolate
func = interpolate.splrep(date_list,velocity_list)

In [ ]:
#Mendapatkan Daftar Gambar Katrina
import os
e = os.listdir('Dataset/tcdat/tc05/ATL/12L.KATRINA/ir/geo/1km')
#Urutkan gambar berdasarkan waktu
e.sort()
x=[]
y=[]
for m in e :
    try :
        #Lepaskan Data Waktu dari gambar dan ubah menjadi tipe waktu-waktu.
        time_img=(datetime.strptime(m[:13], "%Y%m%d.%H%M"))
        #Jika Gambar diambil antara data teks yang tersedia
        if(time_img>=first and time_img <= last):
            #Dapatkan Nilai Interpolasi untuk waktu itu dan Simpan
            value = int(interpolate.splev((time_img-first).total_seconds(),func))
            x.append((time_img-first).total_seconds())
            y.append(value)
    except :
       pass   

In [ ]:
import matplotlib.pyplot as plt
#Plot Semua Poin Data Tersimpan
f = plt.figure(figsize=(24,10))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax.title.set_text('Datapoints frm csv file')
ax2.title.set_text('Interpolated from CSV file to images')
ax.plot(date_list,velocity_list,'-o')
ax2.plot(x,y)

### Sekarang kita telah menginterpolasi dan menemukan kecepatan yang relevan untuk semua gambar di antara kerangka waktu teks yang direkam. Sekarang mari kita gunakan untuk melatih Model kita.

# Penggabungan :

### Mempersiapkan Dataset

#####  Semua modul di atas digabungkan menjadi satu dan menjadikannya satu fungsi untuk memuat data

In [ ]:
import sys
sys.path.append('/workspace/python/source_code')
# Import Fungsi Utlility
from utils import * 
# Muat kumpulan data
filenames,labels = load_dataset()

In [ ]:
val_filenames , val_labels = make_test_set(filenames,labels,val=0.1)

# Memahami Dataset :

Kita bisa melihat baris berikut dari Output: 

`[344, 344, 344, 344, 344, 344, 344, 344]` and `{2: 7936, 3: 5339, 1: 3803, 4: 2934, 5: 2336, 6: 2178, 7: 204, 0: 100}`

Ini adalah distribusi set validasi dan set pelatihan kita disetiap kelas-kelasnya.

Untuk set validasi kita menggunakan set *Stratified Validation* sehingga set validasi kita hampir mewakili seluruh kelas.


In [ ]:
#Membuat train test set
test = 0.2
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(filenames, labels, test_size=test, random_state=1)

# One-Hot Encoding 

`y_train` adalah daftar yang berisi data dari 0-7 seperti [ 2,4,5,....] tetapi Model kita Membutuhkan Input Array untuk Setiap Output sebagai vektor 1D :

2 --- > [ 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0] 

4 --- > [ 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0] 


Ini dikodekan seperti itu karena menjaga nilai-nilai lain 0 diperlukan untuk model untuk menemukan Model Rugi dan menggunakan backpropagation untuk membuatnya mempelajari _Weight Matrix_.

Gambar yang diberikan di bawah ini adalah contoh One-Hot Encoding:

![alt text](images/one_hot.jfif)

Sumber: [What is One Hot Encoding and How to Do It](https://medium.com/@michaeldelsole/what-is-one-hot-encoding-and-how-to-do-it-f0ae272f1179)

In [ ]:
import tensorflow as tf
y_train = tf.one_hot(y_train,depth=8)
y_test = tf.one_hot(y_test,depth=8)
val_labels = tf.one_hot(val_labels,depth=8)

In [ ]:
train,test,val = make_dataset((x_train,y_train,128),(x_test,y_test,32),(val_filenames,val_labels,32))

### Mendefinisikan Model

![alt_text](images/model.png)

Kita akan Mengimplementasikan model ini di Keras menggunakan kode berikut

In [ ]:
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf.random.set_seed(1337)

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten ,Dropout, MaxPooling2D
from tensorflow.keras import backend as K 

#Setel Ulang Grafik dan Buat model Berurutan
K.clear_session()
model = Sequential()
#Lapisan Konvolusi

model.add(Conv2D(64, kernel_size=10,strides=3, activation='relu', input_shape=(232,232,3)))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Conv2D(256, kernel_size=5,strides=1,activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Conv2D(288, kernel_size=3,strides=1,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=1))
model.add(Conv2D(272, kernel_size=3,strides=1,padding='same',activation='relu'))
model.add(Conv2D(256, kernel_size=3,strides=1,activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Dropout(0.5))
model.add(Flatten())

#Lapisan Linier

model.add(Dense(3584,activation='relu'))
model.add(Dense(2048,activation='relu'))
model.add(Dense(8, activation='softmax'))

#Ringkasan Model Cetak

model.summary()

### Menyusun dan Melatih Model

Kita akan menggunakan yang berikut ini: 

- Pengoptimal : SGD ( Stochastic Gradient Descent ) dengan parameter yang disebutkan dalam makalah penelitian.
    - Learning Rate : 0.001
    - Momentum : 0.9
- Loss Function : Categorical Cross Entropy (Digunakan dalam klasifikasi multi-kelas)
- Metrik : Kita akan menggunakan dua metrik untuk menentukan performa model kita 
    - Akurasi : Jumlah Prediksi yang benar / Total jumlah Prediksi
    - Akurasi -2 Teratas : Akurasi 2 Teratas berarti bahwa salah satu dari model 2 jawaban probabilitas tertinggi Anda harus cocok dengan jawaban yang diharapkan.

In [ ]:
import functools

# Sertakan Metrik Akurasi Top-2
top2_acc = functools.partial(tensorflow.keras.metrics.top_k_categorical_accuracy, k=2)
top2_acc.__name__ = 'top2_acc'

# Tentukan Jumlah epochs
epochs = 4

#Tapi Melatih model kita dari awal akan memakan waktu lama
#Jadi kita akan memuat model yang dilatih sebagian untuk mempercepat proses 
model.load_weights("trained_16.h5")

#Pengoptimal
sgd = tensorflow.keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9)


#Kompilasi Model dengan Loss Function , Pengoptimal dan Metrik
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, 
              optimizer=sgd,
              metrics=['accuracy',top2_acc])

#Latih Modelnya
trained_model = model.fit(train,
          epochs=epochs,
          verbose=1,
          validation_data=val)

#Uji Model Terhadap Set Validasi
score = model.evaluate(test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Visualisasi

Sekarang mari kita memvisualisasikan bagaimana model kita bekerja selama proses pelatihan: 

In [ ]:
import matplotlib.pyplot as plt
f = plt.figure(figsize=(15,5))
ax = f.add_subplot(121)
ax.plot(trained_model.history['accuracy'])
ax.plot(trained_model.history['val_accuracy'])
ax.set_title('Model Accuracy')
ax.set_ylabel('Accuracy')
ax.set_xlabel('Epoch')
ax.legend(['Train', 'Val'])

ax2 = f.add_subplot(122)
ax2.plot(trained_model.history['loss'])
ax2.plot(trained_model.history['val_loss'])
ax2.set_title('Model Loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
ax2.legend(['Train', 'Val'],loc= 'upper left')

plt.show()

## Confusion Matrix :

Sebuah Confusion matrix adalah tabel yang sering digunakan untuk menggambarkan kinerja model klasifikasi (atau "classifier") pada satu set data uji yang nilai sebenarnya diketahui.

Di sini, baris menampilkan kelas yang diprediksi dan kolom adalah nilai kebenaran dari kelas. Dari sini kita dapat memperkirakan bagaimana model kita bekerja pada kelas yang berbeda yang pada gilirannya akan membantu kita menentukan bagaimana data kita harus dimasukkan ke dalam model kita.




In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
import pandas as pd

#Merencanakan sebuah heatmap menggunakan confusion matrix
pred = model.predict(val)
p = np.argmax(pred, axis=1)
y_valid = np.argmax(val_labels, axis=1, out=None)
results = confusion_matrix(y_valid, p) 
classes=['NC','TD','TC','H1','H3','H3','H4','H5']
df_cm = pd.DataFrame(results, index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (15,15))

sn.heatmap(df_cm, annot=True, cmap="Blues")

### Selamat menjalankan model pertama Anda. Sekarang Di notebook berikutnya, mari kita coba memahami kekurangan model ini dan membuatnya lebih baik:


Kita dapat melihat bahwa akurasi validasi lebih rendah daripada akurasi pelatihan. Ini karena modelnya tidak diregulasi dengan benar dan kemungkinan alasannya adalah: 

**Poin data tidak cukup/ kelas tidak seimbang**

Dengan menggunakan teknik yang berbeda, kita akan mengatur dan menormalkan model di notebook yang akan datang.

## Penting:
<mark>Matikan kernel sebelum mengklik "Notebook Berikutnya" untuk mengosongkan memori GPU.</mark>

## Lisensi:
Materi ini dirilis oleh OpenACC-Standard.org, bekerja sama dengan NVIDIA Corporation, di bawah Creative Commons Attribution 4.0 International (CC BY 4.0).

[Previous Notebook](Approach_to_the_Problem_&_Inspecting_and_Cleaning_the_Required_Data.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[1](The_Problem_Statement.ipynb)
[2](Approach_to_the_Problem_&_Inspecting_and_Cleaning_the_Required_Data.ipynb)
[3]
[4](Countering_Data_Imbalance.ipynb)
[5](Competition.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](Countering_Data_Imbalance.ipynb)

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../Start_Here.ipynb)